In [1]:
import ujson as json
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly.plotly as py
import datetime
from __future__ import division

from moztelemetry import get_pings, get_pings_properties, get_one_ping_per_client

%pylab inline

Unable to parse whitelist (/home/hadoop/anaconda2/lib/python2.7/site-packages/moztelemetry/bucket-whitelist.json). Assuming all histograms are acceptable.
Populating the interactive namespace from numpy and matplotlib


In [21]:
# We grab pings starting from 2 days ago until [TimeWindow] days ago.
TimeWindow = 14

# Additional filters.
Filters = {
    'app': 'Firefox',
    
    # We sample 0.5% of pings. For testing, it is beter to use a small number
    # here (like 0.001) to speed up processing time.
    'fraction': 0.005,
    
    # Optionally restrict pings to a single channel.
    # 'channel': 'beta',
}

# In case you need them!
VendorIDs = {
    'Intel': '0x8086',
    'NVIDIA': '0x10de',
    'AMD': '0x1002',
    'ATI': '0x1002'
}

In [22]:
###############################
# This section gathers pings. #
###############################

def run_get_pings():
    def fmt_date(d):
        return d.strftime("%Y%m%d")
    t1 = fmt_date(datetime.datetime.now() - datetime.timedelta(TimeWindow + 2)) # go back 16 days
    t2 = fmt_date(datetime.datetime.now() - datetime.timedelta(2)) # go back 2 days
    return get_pings(sc, build_id=(t1, t2), **Filters)

# Get pings for the parameters in the previous step.
raw_pings = run_get_pings()

In [23]:
######################################################################
# This section takes the raw ping list, then formats and filters it. #
######################################################################

# Map the pings into a more readable dictionary-like form. To see
# what these look like, execute "pings.take(1)".
unique_pings = get_pings_properties(raw_pings, [
  "clientId",
  "environment/build/version",
  "environment/system/os/name",
  "environment/system/os/version",
  "environment/system/os/servicePackMajor",
  "environment/system/os/servicePackMinor",
  "environment/system/gfx/adapters",
  "payload/histograms/CANVAS_WEBGL_SUCCESS"
])
unique_pings = get_one_ping_per_client(unique_pings)

In [24]:
# We add two extra steps. The first rewrites the ping to have some
# information more easily accessible (like the primary adapter),
# and the second step removes any pings that don't have adapter
# information.
def rewrite_ping(p):
    adapters = p.get('environment/system/gfx/adapters', None)
    if not adapters:
        return None
    adapter = adapters[0]
            
    p['adapter'] = adapter
            
    # Convert the version to a tuple of integers.
    if 'driverVersion' in adapter:
        p['driverVersion'] = [int(n) for n in adapter['driverVersion'].split('.') if n.isdigit()]
    return p

def filter_ping(p):
    return 'adapter' in p
pings = unique_pings.map(rewrite_ping).filter(filter_ping)
pings = pings.cache()

#pings = pings.cache()

In [25]:
# Observe the format of a random ping. This may take some time since it has to
# execute the pipeline.
#pings.take(1)
#pings.take(1)
#new_pings = pings.map(rewrite_ping).filter(filter_ping)
pings.take(1)

[{'adapter': {u'GPUActive': True,
   u'description': u'Intel(R) G33/G31 Express Chipset Family',
   u'deviceID': u'0x29c2',
   u'driver': u'igxprd32',
   u'driverDate': u'1-13-2010',
   u'driverVersion': u'6.14.10.5218',
   u'subsysID': u'29c28086',
   u'vendorID': u'0x8086'},
  'clientId': u'59274342-94fb-416b-abcd-b37ad53114fc',
  'driverVersion': [6, 14, 10, 5218],
  'environment/build/version': u'46.0',
  'environment/system/gfx/adapters': [{u'GPUActive': True,
    u'description': u'Intel(R) G33/G31 Express Chipset Family',
    u'deviceID': u'0x29c2',
    u'driver': u'igxprd32',
    u'driverDate': u'1-13-2010',
    u'driverVersion': u'6.14.10.5218',
    u'subsysID': u'29c28086',
    u'vendorID': u'0x8086'}],
  'environment/system/os/name': u'Windows_NT',
  'environment/system/os/servicePackMajor': 3,
  'environment/system/os/servicePackMinor': 0,
  'environment/system/os/version': u'5.1',
  'payload/histograms/CANVAS_WEBGL_SUCCESS': None}]

In [26]:
# Count the total number of sessions in the dataset.
TotalSessions = pings.count()
print('Number of sessions: {0}'.format(TotalSessions))

Number of sessions: 236783


In [27]:
import json
import urllib2

gpu_db = json.load(urllib2.urlopen('https://raw.githubusercontent.com/jrmuizel/gpu-db/master/intel.json'))
devices = {}
for gen in gpu_db['8086'].items():
    for chipset in gen[1].items():
        for dev in chipset[1]:
            #print dev, gen[0]
            devices[int(dev,16)] = gen[0]

In [36]:
# Sample filter #1 - how many people are using Intel devices
# with a driver less than 8.15.10.2622? (bug 1175366).
def sample_filter_1(p):
    if not 'vendorID' in p['adapter']:
        return False
    if not 'driver' in p['adapter']:
        return False
    if p['adapter']['vendorID'] != VendorIDs['Intel']:
        return False
    if p['payload/histograms/CANVAS_WEBGL_SUCCESS'] is None:
        return False
    if devices[int(p['adapter']['deviceID'],16)] in ('gen6'):
        return True

    #if devices[int(p['adapter']['deviceID'],16)] in ('gen7'):

     #   return True
    return False

interesting_pings = pings.filter(sample_filter_1)
def webgl_fail(p):
    if p['payload/histograms/CANVAS_WEBGL_SUCCESS'][0] > 0:
        return True

     #   return True
    return False
fail_pings = interesting_pings.filter(webgl_fail)

def vga_fail(p):
    if 'vga' in p['adapter']['driver']:
        return True


     #   return True
    return False
vga_pings = fail_pings.filter(vga_fail)

def not_vga_fail(p):
    if 'vga' in p['adapter']['driver']:
        return False
     #   return True
    return True
not_vga_pings = fail_pings.filter(not_vga_fail)


#TDRResults = sample_result_1.map(lambda p: p[DeviceResetReasonKey]).reduce(lambda x, y: x + y)
#help(pd.Series(TDRResults.values, DeviceResetReason))
#print(pd.Series(TDRResults.values[1:8],DeviceResetReason))
#pd.Series(TDRResults.values[1:8],DeviceResetReason).plot.bar()
#reduce(lambda x, y: sample_result_1.take(2))
#sample_result_1.take(1)[0]['payload/histograms/DEVICE_RESET_REASON'])
#fail_pings.take(1)
def match_count(a, b):
    print('{0} out of {1} sessions matched. ({2:.2f}%)'.format(
        a.count(),
        b.count(),
       ((a.count() / b.count()) * 100)))



def map_x_to_count(data, sourceKey):
    def extract(p):
        return (p['adapter'][sourceKey],)
    return data.map(extract).countByKey()    
match_count(interesting_pings, pings)    
match_count(fail_pings, interesting_pings)
match_count(vga_pings, fail_pings)
map_x_to_count(not_vga_pings, 'driverVersion')
#not_vga_pings.take(5)

820 out of 236783 sessions matched. (0.35%)
64 out of 820 sessions matched. (7.80%)
24 out of 64 sessions matched. (37.50%)


defaultdict(int,
            {u'6.14.10.5337': 1,
             u'6.14.10.5361': 1,
             u'8.15.10.2253': 1,
             u'8.15.10.2266': 1,
             u'8.15.10.2279': 1,
             u'8.15.10.2321': 1,
             u'8.15.10.2342': 1,
             u'8.15.10.2345': 1,
             u'8.15.10.2353': 1,
             u'8.15.10.2361': 2,
             u'8.15.10.2418': 2,
             u'8.15.10.2622': 2,
             u'8.802.1.0': 1,
             u'8.810.4.3000': 1,
             u'8.810.4.5000': 1,
             u'8.830.7.0': 1,
             u'8.840.7.0': 3,
             u'8.840.7.1000': 1,
             u'8.840.7.1100': 2,
             u'8.840.7.3000': 1,
             u'8.840.7.9000': 2,
             u'8.861.0.0': 1,
             u'8.862.4.0': 1,
             u'8.882.2.0': 2,
             u'8.882.2.3000': 2,
             u'8.901.1.1000': 1,
             u'8.930.0.0': 1,
             u'8.933.2.0': 1,
             u'8.981.0.0': 1,
             u'9.0.0.0': 1,
             u'9.17.10.33